# Credit Risk Analysis

*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

The task of risk analysis is to access the risk of given assets facing the fluctuations of financial systems, i.e. the Value at Risk (VaR). Such quantity can measure the potent loss of given assets in a financial system. There are many factors determining the Value of Risk of assets, including the favor of risk for different financial organizations, the characteristics of system fluctuations and the risk-related properties of assets. In 2014 Rutkowski and Tarca [1] provided a mathematical model that estimates the risk measure. This model suggests that the whole financial system can be modelled by a Brownian motion process, so that the minimum upper bound of assets' loss can be computed under an appropriate confidence level.

Credit risk analysis is a direct application of such model, which can evaluate the regulatory capital for credit risk in banking industry i.e. the VaR of credit asset combination held under the current financial . In this model, we denote the given asset combination as $\{ 0, ..., K-1 \}$, and the risk measures of the total loss as
$$
\mathcal{L} = \sum_{k=0}^{K - 1} \lambda_k X_k(Z)
.$$
Here, $\lambda_k$ is the loss given default of the $k$-th asset; $Z$ is the standard Gaussian variable that implicitly models the process of financial system; $X_k(Z)$ is the Bernoulli variable that models the default process of the $k$-th asset. In particular, when $Z = z$, the parameter $p_k(z)$ of random variable $X_k(Z)$ i.e. the default probability of the $k$-th asset, is dependent on the basic default probability without the effect of financial system $p_k^{(0)} \in [0, 1]$, and the asset sensitivity to the system risk $\rho_k \in [0, 1]$. After settling the confidence level, the Value at Risk in the problem of credit risk analysis shall be defined as "the minimum upper bound of credit asset combination under confidence level $\alpha$". That is,
$$
\textrm{VaR}_\alpha(\mathcal{L}) := \inf \{ x \,|\, \textrm{Pr}(\mathcal{L} \leq x) \geq \alpha \}
.$$
For example, if the VaR of a combination of credit assets held by a bank happens to be one million under confidence level $99\%$, then the probability that the total loss given default of such combination is greater than one million is less than $1\%$. In classical calculations, the VaR in the problem of credit risk analysis can be estimated by classical Monte Carlo and bisection search methods:
1. Choose an appropriate VaR guess $\check{x}$ according to the properties of assets.
2. Use classical Monte Carlo method to estimate the probability $p_{\check{x}} = \textrm{Pr}(\mathcal{L} \leq \check{x})$.
3. Compare the probability with $\alpha$, and update the VaR guess $\check{x}$ according to the comparison result and bisection search method.
4. If the convergence criteria is met, output $\check{x} = \textrm{VaR}_\alpha(\mathcal{L})$; otherwise return to step $2$.

## Quantum solution

Unlike classical algorithms, quantum computation can use the algorithm of quantum amplitude estimation (QAE) to replace the classical Monte Carlo method in the second step mentioned above, to enhance the efficiency of probability estimation. Through the characteristics of quantum superposition and entanglement, such quantum scheme is expected to obtain the advantage of quadratic acceleration compared with classical schemes [2]. Next, we will show how to use Paddle Quantum to simulate this quantum scheme to complete the VaR computational problem of credit risk analysis.

### Online demonstration

We have set a parameter that can be used directly for the VaR computation of asset combinations. Just configure it in the configuration file `config.toml` and enter the command 
`python credit_risk.py --config config.toml`.
The VaR of configured assets is then be computed.

Here, we give a version of the online demo that can be tested online. First define the contents of the configuration file:

In [1]:
cra_toml = r"""
# The config for credit risk analysis model
# number of assets.
num_assets = 4
# basic default probabilities.
base_default_prob = [0.15, 0.25, 0.39, 0.58]
# sensitivity.
sensitivity = [0.37, 0.21, 0.32, 0.02]
# loss given default.
lgd = [5, 1, 3, 4]
# confidence level.
confidence_level = 0.99
# degree of simulation. Higher the degree, preciser the simulation.
degree_of_simulation = 4
"""

The financial module of the Paddle Quantum enables numerical simulation of quantum amplitude estimation scheme. We can import ``CreditRiskAnalyzer`` from the ``paddle_quantum.finance`` module to solve the configured VaR computational problem.

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import toml
from paddle_quantum.finance import CreditRiskAnalyzer

config = toml.loads(cra_toml)
num_assets = config["num_assets"]
base_default_prob = config["base_default_prob"]
sensitivity = config["sensitivity"]
lgd = config["lgd"]
confidence_level = config["confidence_level"]
degree_of_simulation = config["degree_of_simulation"]

estimator = CreditRiskAnalyzer(num_assets, base_default_prob, sensitivity, lgd, confidence_level, degree_of_simulation)
print("The Value at Risk of these assets are", estimator.estimate_var())

-----------------------------------------------------------------------------------
Begin bisection search for VaR with confidence level >= 99.0%.
-----------------------------------------------------------------------------------
Lower guess: level            Middle guess: level            Upper guess: level    
    -1     : 0.000                  6    : 0.691                 13     : 1.000    
     6     : 0.691                  9    : 0.941                 13     : 1.000    
     9     : 0.941                 11    : 0.962                 13     : 1.000    
    11     : 0.962                 12    : 0.990                 13     : 1.000    
-----------------------------------------------------------------------------------
Estimated VaR is 12 with confidence level 99.0%.
-----------------------------------------------------------------------------------
The Value at Risk of these assets are 12


___

# Note

The model presented here is only intended to solve the credit risk analysis problem of a particular model.

# Citation

```
@article{rutkowski2015regulatory,
  title={Regulatory capital modeling for credit risk},
  author={Rutkowski, Marek and Tarca, Silvio},
  journal={International Journal of Theoretical and Applied Finance},
  volume={18},
  number={05},
  pages={1550034},
  year={2015},
  publisher={World Scientific}
}

@article{egger2020credit,
  title={Credit risk analysis using quantum computers},
  author={Egger, Daniel J and Guti{\'e}rrez, Ricardo Garc{\'\i}a and Mestre, Jordi Cahu{\'e} and Woerner, Stefan},
  journal={IEEE Transactions on Computers},
  volume={70},
  number={12},
  pages={2136--2145},
  year={2020},
  publisher={IEEE}
}
```

# References

[1] Rutkowski, Marek, and Silvio Tarca. "Regulatory capital modeling for credit risk." International Journal of Theoretical and Applied Finance 18.05 (2015): 1550034.

[2] Egger, Daniel J., et al. "Credit risk analysis using quantum computers." IEEE Transactions on Computers 70.12 (2020): 2136-2145.